In [1]:
import numpy as np
from random import random
import pandas as pd

In [2]:
class MLP(object):
    
    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):
        
        
        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs
        
        layers = [num_inputs] + hidden_layers + [num_outputs]
        
        weights = []
        for i in range(len(layers)-1):
            w = np.random.rand(layers[i], layers[i+1])
            weights.append(w)
        self.weights = weights
        
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives
        
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations
        
    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals.
        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
            
        """
        
#         import pdb
#         pdb.set_trace()
        


        # the input layer activation is just the input itself
        activations = inputs
        self.activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.activations[i + 1] = activations

        # return output layer activation
        return activations
    def forward_propagate_test(self, inputs):
        """Computes forward propagation of the network based on input signals.
        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
            
        """
        
#         import pdb
#         pdb.set_trace()

        # the input layer activation is just the input itself
        activations = inputs
        self.activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.activations[i + 1] = activations

        # return output layer activation
        return activations
    
    def back_propagate(self, error):
        """Backpropogates an error signal.
        Args:
            error (ndarray): The error to backprop.
        Returns:
            error (ndarray): The final error of the input
        """

        # iterate backwards through the network layers
        for i in reversed(range(len(self.derivatives))):

            # get activation for previous layer
            activations = self.activations[i+1]

            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)

            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T

            # get activations for current layer
            current_activations = self.activations[i]

            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations, delta_re)

            # backpropogate the next error
            error = np.dot(delta, self.weights[i].T)
            
    def train(self, inputs, targets, epochs, learning_rate):
        """Trains model running forward prop and backprop
        Args:
            inputs (ndarray): X
            targets (ndarray): Y
            epochs (int): Num. epochs we want to train the network for
            learning_rate (float): Step to apply to gradient descent
        """
        # now enter the training loop
        for i in range(epochs):
            sum_errors = 0

            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]

                # activate the network!
                output = self.forward_propagate(input)

                error = target - output

                self.back_propagate(error)

                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)

                # keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)

            # Epoch complete, report the training error
            print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))

        print("Training complete!")
        print("=====")
    
    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        
        y = 1.0 / (1 + np.exp(-x))
        return y
    
    
    def gradient_descent(self, learningRate=1):
        """Learns by descending the gradient
        Args:
            learningRate (float): How fast to learn.
        """
        # update the weights by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate
            
    def _sigmoid_derivative(self, x):
        """Sigmoid derivative function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        return x * (1.0 - x)
    
    def _mse(self, target, output):
        """Mean Squared Error loss function
        Args:
            target (ndarray): The ground trut
            output (ndarray): The predicted values
        Returns:
            (float): Output
        """
        return np.average((target - output) ** 2)
    
    
    
    def create_seed(self):
        
        
        
        self.seed_weights = None
        self.seed_derivatives = None
        self.seed_activations = None
        
        self.seed_weights = self.weights[:-2]
        self.seed_derivatives = self.derivatives[:-2]
        self.seed_activations = self.activations[:-1]

        
    
    #Prime base network
    
    def prime_base_network(self, inputs, targets, cycles, learning_rate = 0.5):
        
        self.train(inputs, targets, cycles, 0.5)
        
    def remove_temp_classifier(self):

        
        self.weights.pop()
        self.derivatives.pop()
        self.activations.pop()
    
    def add_destination_layer(self):
        w = np.random.rand(self.weights[-1].shape[0], self.weights[-1].shape[1])
        self.weights.append(w)
        
        d = np.zeros((self.derivatives[-1].shape[0], self.derivatives[-1].shape[1]))
        self.derivatives.append(d)
        
        a = np.zeros(len(self.activations[-1]))
        self.activations.append(a)
    
    def add_class_layer_final(self, targets, shape):
        
#         import pdb
#         pdb.set_trace()
        
        num_classes = len(np.unique(targets))
        w = np.random.rand(shape, num_classes)
        self.weights.append(w)
    
        d = np.zeros((shape, num_classes))
        self.derivatives.append(d)
        
        class_activation = np.zeros(num_classes)
        self.activations.append(class_activation)
    
    
    def add_class_layer(self, targets):
        
#         import pdb
#         pdb.set_trace()
        
        num_classes = len(np.unique(targets))
        w = np.random.rand(self.weights[-1].shape[0], num_classes)
        self.weights.append(w)
    
        d = np.zeros((self.derivatives[-1].shape[0], num_classes))
        self.derivatives.append(d)
        
        class_activation = np.zeros(num_classes)
        self.activations.append(class_activation)
    
    def create_temp_classifier_seed(self, targets):
        
#         import pdb
#         pdb.set_trace()
        
        num_classes = len(np.unique(targets))
        w = np.random.rand(self.seed_weights[-1].shape[0], num_classes)
        self.seed_weights.append(w)
        
        d = np.zeros((self.seed_derivatives[-1].shape[0], num_classes))
        self.seed_derivatives.append(d)
        
        class_activation = np.zeros(num_classes)
        self.seed_activations.append(class_activation)
    
    def forward_propagate_seed(self, inputs):
        """Computes forward propagation of the network based on input signals.
        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
        """

        # the input layer activation is just the input itself
        activations = inputs
        self.seed_activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.seed_weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.seed_activations[i + 1] = activations

        # return output layer activation
        return activations
    
    def back_propagate_seed(self, error):
        """Backpropogates an error signal.
        Args:
            error (ndarray): The error to backprop.
        Returns:
            error (ndarray): The final error of the input
            
        """

#         import pdb
#         pdb.set_trace()

        # iterate backwards through the network layers
        for i in reversed(range(len(self.seed_derivatives))):

            # get activation for previous layer
            activations = self.seed_activations[i+1]

            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)

            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T

            # get activations for current layer
            current_activations = self.seed_activations[i]

            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # save derivative after applying matrix multiplication
            self.seed_derivatives[i] = np.dot(current_activations, delta_re)

            # backpropogate the next error
            error = np.dot(delta, self.seed_weights[i].T)
    
    def train_seed(self, inputs, targets, epochs, learning_rate = 0.5):
        """Trains model running forward prop and backprop
        Args:
            inputs (ndarray): X
            targets (ndarray): Y
            epochs (int): Num. epochs we want to train the network for
            learning_rate (float): Step to apply to gradient descent
        """
        # now enter the training loop
        for i in range(epochs):
            sum_errors = 0

            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]

                # activate the network!
                output = self.forward_propagate_seed(input)

                error = target - output

                self.back_propagate_seed(error)

                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)

                # keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)

            # Epoch complete, report the training error
            #print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))

        print("Training complete!")
        print("=====")
        
    def prime_seed_network(self, inputs, targets, cycles, learning_rate = 0.5):
        
        self.train_seed(inputs, targets, cycles, 0.5)
        
    def remove_temp_classifier_seed(self):
        self.seed_weights.pop()
        self.seed_derivatives.pop()
        self.seed_activations.pop()
    
    def extreme_member_classes(self, inputs, targets):
        
        #Create a temporary seed matrix
        self.create_seed()
        #Attatch a temp Classification layer
        self.create_temp_classifier_seed(targets)
        #Prime Seed 
        self.prime_seed_network(inputs, targets, cycles = 10)
        #remove the temporary classifier
        self.remove_temp_classifier_seed()
        #for each class in the dataset:
        
        Classes = np.unique(targets)
        sorted_classes_list = []
        #Loop through the classes in the dataset
        for i in Classes:
            count = targets.tolist().count(i)  #The number of class members 
            sum_perceptron = []
            avg_perceptron = []
            
            #initialize the perceptrons
            for j in range(len(self.seed_weights[-1])):
                sum_perceptron.append(0)
            #for each member in the dataclass forward propogate
            filter_indices = np.where(targets == i)
            Class_inputs = np.take(inputs, filter_indices, axis = 0)
            for inp in Class_inputs[0]: 
                output_single = self.forward_propagate_seed(inp)
                #find the outputs at the last layer
                for k in range(len(self.seed_weights[-1])):
                    sum_perceptron[k] += output_single[k]
            for s in sum_perceptron:
                avg_perceptron.append(s/count)
            Error_list = []
            #Find the avg - individual_output -> Error
            for inp in Class_inputs[0]:
                err = 0
                output_single = self.forward_propagate_seed(inp)
                for o in range(len(output_single)):
                    err += avg_perceptron[o] - output_single[o]
                Error_list.append(err)
            sort_indices = np.argsort(Error_list)
            #sort the class members according to errors. The first and the last members of the sorted list would be termed as extreme members
            Class_sorted = Class_inputs[0][sort_indices]
            sorted_classes_list.append(Class_sorted)
        return sorted_classes_list
    
    def return_source_activations(self):
        #return activations of the source layer
        return self.activations[-2]
    
    def return_acc(self, inputs, targets):
        pred = []
        for i in inputs:
            output = self.forward_propagate(i)
            pred.append(np.argmax(output))
        from sklearn.metrics import accuracy_score
        acc = accuracy_score(targets, pred)
        print(pred)
        return acc


In [3]:
###Alternate### This works better now

In [4]:
def ANG(inputs, targets):
    #initialize the multilayer perceptron to take 2 inputs and generate a binary classifier
    mlp = MLP(2, [3, 3], 2)
    #Prime the base network for 10 epochs
    mlp.prime_base_network(inputs, targets, cycles = 10)
    #Strip the network of the partially fitted classifier
    mlp.remove_temp_classifier()
    #Add an empty destination layer
    mlp.add_destination_layer()
    #Add an untrained classification layer
    mlp.add_class_layer(targets)
    accuracy = 0
    items = inputs
    test_targets = targets
    percep = 0
    
    #Place a stopping value on accuracy
    while accuracy<0.9:
        #Returns a list of n lists, where n is the number of classes in the dataset. Each of the inner lists consists of all the members of a particular class, arranged in order of the distance of their individual output from the average output
        sorted_classes = mlp.extreme_member_classes(inputs, targets)
        mlp.remove_temp_classifier()
        
        for i in sorted_classes:
            #Loop through each class and identify the extreme members
            extremes = []
            extremes.append(i[0])
            extremes.append(i[-1])
            #Loop through the extreme members
            for ext in extremes:
                #Propogate each extreme member through the network
                op = mlp.forward_propagate_test(ext)
                #Pull out the outputs at the source layer
                fp_output = mlp.return_source_activations()
                sum_op = 0
                #Find the sum of the outputs
                for s in fp_output:
                    sum_op += s
                #Find the average of the outputs
                average = np.average(fp_output)
                sum_avg = 0
                for s in fp_output:
                    sum_avg += (average - s) * (average - s)
                sd = np.std(fp_output)
                x = 1 # X is the number of standard deviation units at which to set the threshold
                try:
                    #Try and see if there already exists a neuron to which to map the extreme member's output
                    mlp.activations[-1][percep] = 0
                except:
                    #If no neuron is present, create a new one by initialzing new weights, derivatives and activations
                    temp_weights = []
                    for w in range(len(mlp.weights[-1])):
                        print(mlp.weights[-1][w])
                        temp_weights.append(np.append(mlp.weights[-1][w],np.random.rand()))
                    mlp.weights[-1] = np.array(temp_weights)
                    temp_derivatives = []
                    for d in range(len(mlp.derivatives[-1])):
                        temp_derivatives.append(np.append(mlp.derivatives[-1][d],0))
                    mlp.derivatives[-1] = np.array(temp_derivatives)
                    mlp.activations[-1] = np.append(mlp.activations[-1], 0)
                    
                percep = percep + 1
                
                #Check if each of the source layer outputs meet the threshold
                for conn in range(len(fp_output)):
                    #If they do, they are termed as critical connections and the weights are preserved
                    if fp_output[conn] >= x*sd or fp_output[conn]<= -x*sd:
                        print("This is a critical connection")
                    else:
                        #If not, the weights are set to 0 and the connection is weakened 
                        print("This is not a critical connection")
        
                        length = len(mlp.weights[-1])
                        for weights in mlp.weights[-1]:
                            weights[conn] = 0
        #Bring back the classification layer and train the model on all the data
        mlp.add_class_layer_final(targets, mlp.weights[-1].shape[1])
        mlp.train(inputs, targets, 50, 0.5)
        #Check if the accuracy meets the threshold
        accuracy = mlp.return_acc(items, test_targets)
        extreme_members = []
        #Remove the extreme class members from the dataset and repeat the process until the target metrics are met
        for i in sorted_classes:
            extreme_members.append(i[0])
            extreme_members.append(i[-1])
        for i in extreme_members:
            ind = np.where(inputs == i)
            inputs = np.delete(inputs, ind[0][0], 0)
            targets = np.delete(targets, ind[0][0])
        print(accuracy)
    print(mlp.weights)
    return mlp
            
        



            
            
        

In [5]:
from sklearn.datasets import make_classification

In [6]:
items, targets = make_classification(n_features=2, n_redundant=0, n_informative=1, n_clusters_per_class=1)

In [7]:
mlp = ANG(items, targets)

Error: 0.2634815972560526 at epoch 1
Error: 0.2499587393403544 at epoch 2
Error: 0.24169899653805857 at epoch 3
Error: 0.21188170994403305 at epoch 4
Error: 0.14156469884679498 at epoch 5
Error: 0.08527363240662984 at epoch 6
Error: 0.06284793693746743 at epoch 7
Error: 0.05367582420920198 at epoch 8
Error: 0.04917555712853198 at epoch 9
Error: 0.04665942709844988 at epoch 10
Training complete!
=====
Training complete!
=====
This is a critical connection
This is a critical connection
This is a critical connection
This is a critical connection
This is not a critical connection
This is a critical connection
This is a critical connection
This is a critical connection
This is a critical connection
[0.74536261 0.         0.0189667 ]
[0.48648922 0.         0.45173339]
[0.34629423 0.         0.23829522]
This is a critical connection
This is not a critical connection
This is a critical connection
Error: 0.25157472868829944 at epoch 1
Error: 0.17489206791666045 at epoch 2
Error: 0.0914466656778

Error: 0.037275528251753426 at epoch 2
Error: 0.023726857708996105 at epoch 3
Error: 0.019387150440076495 at epoch 4
Error: 0.017269771805716504 at epoch 5
Error: 0.01599456559772611 at epoch 6
Error: 0.015113803324373693 at epoch 7
Error: 0.014450660162200289 at epoch 8
Error: 0.013928472913497737 at epoch 9
Error: 0.013508971413270792 at epoch 10
Error: 0.013168568346976 at epoch 11
Error: 0.012889927136115045 at epoch 12
Error: 0.012659255857214067 at epoch 13
Error: 0.012465497364313486 at epoch 14
Error: 0.012300002535151342 at epoch 15
Error: 0.012156222528578812 at epoch 16
Error: 0.012029332972725073 at epoch 17
Error: 0.011915836283833105 at epoch 18
Error: 0.01181320362725188 at epoch 19
Error: 0.011719589315921661 at epoch 20
Error: 0.01163362102849079 at epoch 21
Error: 0.011554253529402642 at epoch 22
Error: 0.0114806695838008 at epoch 23
Error: 0.011412213638385208 at epoch 24
Error: 0.01134834741004106 at epoch 25
Error: 0.011288619859704483 at epoch 26
Error: 0.01123264

In [8]:
mlp.weights


[array([[-0.56272251,  0.14556564,  0.39138218],
        [-5.08753835,  2.47674931,  4.88646242]]),
 array([[ 2.40599   ,  3.3670102 , -4.34982088],
        [-1.89980552, -2.36886296,  2.61605599],
        [-3.01414799, -4.36362055,  5.4877335 ]]),
 array([[ 0.13379099, -0.46327784, -0.24279504, -1.87100905,  1.57072636,
         -1.30901592, -1.15262015,  1.50630533,  0.27193815, -0.6958853 ,
         -0.46073315,  0.28019748,  0.83841207,  0.5315544 ,  0.51940643,
          0.7715446 ],
        [ 0.15250754, -0.51573143, -0.26837527, -3.1307399 ,  1.57013065,
         -1.9193702 , -1.66658975,  1.71689677,  0.43261216,  0.04628637,
          0.11176551, -0.03132994,  1.2672274 ,  0.29479349,  0.95749027,
          0.36570614],
        [-0.18046122,  0.32968199,  0.1282209 ,  4.87710697, -2.12682656,
          2.60236823,  2.30244582, -2.01192666,  0.0696618 ,  1.00215423,
          1.00423716,  0.18587412,  0.05130136,  0.84779043, -0.09476366,
         -0.09362606]]),
 array([[-0.52

In [9]:
mlp.activations

[array([-1.15461518, -1.64272619]),
 array([9.99877486e-01, 1.42493343e-02, 2.07739980e-04]),
 array([0.91513409, 0.96550832, 0.0132441 ]),
 array([0.56642845, 0.28545981, 0.38234413, 0.00928169, 0.94907972,
        0.04668035, 0.06701864, 0.95300161, 0.66093712, 0.3591936 ,
        0.42545436, 0.55690999, 0.87989725, 0.68617503, 0.80195152,
        0.74229268]),
 array([0.01263833, 0.01221807])]